In [1]:
import tuxml
import pandas as pd 

df = tuxml.load_dataset()
df.replace(2, 0, inplace=True)

In [2]:
df.sort_values("vmlinux")[:5]

,OPENVSWITCH,TEXTSEARCH_FSM,NETFILTER_XT_MATCH_TCPMSS,MPLS,NFC_HCI,NETFILTER_XT_MATCH_TIME,NET_MPLS_GSO,NFC_SHDLC,NETFILTER_XT_MATCH_U32,OPENVSWITCH_GRE,...,LZO-vmlinux,LZO,LZ4-bzImage,LZ4-vmlinux,LZ4,cid,nbyes,nbno,nbmodule,nbyesmodule
109097,0,0,0,0,0,0,0,0,0,0,...,2863640,644621,814544.0,2900824,669288,124978,25,9210,0,25
58621,0,0,0,0,0,0,0,0,0,0,...,2777624,558712,724432.0,2810712,578375,74458,24,9211,0,24
107213,0,0,0,0,0,0,0,0,0,0,...,3953176,1734208,1986512.0,4072280,1841339,123091,525,8041,669,1194
99232,0,0,0,0,0,0,0,0,0,0,...,4248440,2027905,2290640.0,4375728,2144007,115110,420,8104,711,1131
99777,0,0,0,0,0,0,0,0,0,0,...,4096536,1877790,2139088.0,4223832,1994471,115655,505,8324,406,911


In [3]:
df_importance = pd.read_csv("feature_importanceRF.csv", header=None)
df_importance.rename(columns={0 : 'option', 1 : 'importance' }, inplace=True)
# df_importance = df[df_importance[:100]["Unnamed: 0"].values]
df_importance = df[df_importance["option"].values] # [:100]
df_importance.head()
# df_importance

,DEBUG_INFO,nbyes,DEBUG_INFO_REDUCED,DEBUG_INFO_SPLIT,X86_NEED_RELOCS,RANDOMIZE_BASE,UBSAN_SANITIZE_ALL,KASAN_OUTLINE,KASAN,UBSAN_ALIGNMENT,...,RSI_DEBUGFS,CRYPTO_RNG_DEFAULT,CRYPTO_ECB,CRYPTO_WORKQUEUE,HISAX_HFC_SX,USB_GSPCA_VC032X,HISAX_ENTERNOW_PCI,CRYPTO_HASH2,HISAX_DEBUG,IP_NF_ARP_MANGLE
14758,0,1219,0,0,1,1,0,1,1,0,...,0,1,1,1,0,0,0,1,0,0
14759,0,1166,0,0,0,0,0,0,0,1,...,0,1,1,1,0,0,0,1,0,0
14760,0,1410,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,1,0,0
14761,0,1924,0,0,0,0,0,0,0,1,...,0,1,1,1,0,0,0,1,0,0
14762,0,2435,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,1,0,0


In [4]:
size_methods = ["vmlinux", "GZIP-bzImage", "GZIP-vmlinux", "GZIP", "BZIP2-bzImage", 
              "BZIP2-vmlinux", "BZIP2", "LZMA-bzImage", "LZMA-vmlinux", "LZMA", "XZ-bzImage", "XZ-vmlinux", "XZ", 
              "LZO-bzImage", "LZO-vmlinux", "LZO", "LZ4-bzImage", "LZ4-vmlinux", "LZ4"]


In [5]:
additional_ft = ["nbyes", "nbno", "nbmodule", "nbyesmodule"]
# very ad-hoc, not consistent with feature importance
# df.drop(columns=additional_ft, inplace=True) # should be a parameter of the experiment (whether or not we keep nbyes, etc.)

In [6]:
#learning a model by using linear regression
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import ElasticNet

def run_regressorML(reg, test_size, size_target, x_train, x_test):
    assert(size_target in size_methods)
    reg.fit(x_train, y_train)
    y_pred = reg.predict(x_test)
    #y_pred = reg.intercept_ + np.sum(reg.coef_ * x_test.values, axis=1)
    dfErrors = pd.DataFrame({"error":(y_pred - y_test).abs(), "% error":((y_pred - y_test)/y_test).abs()*100})
    return dfErrors["% error"].describe()

In [7]:
def ft_importances(coef_, col):
    importanceSeries = pd.Series(coef_, index=col.values)
    return importanceSeries[importanceSeries != 0].abs().sort_values(ascending = False)

In [8]:
df_result  = pd.DataFrame(columns = ['algorithm_name', 'ft_dummyfication', 'origin_ft_selection', 'ft_selection', 'hyperparameters', 'size_target', 'additional_ft', 'test_size', 'coef_order', 'accuracy'])

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNet
from sklearn import linear_model
from sklearn import metrics

target = "vmlinux"
paramsLasso = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
               'max_iter': [1, 5, 10, 100, 1000],}
paramsRidge = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01]}
paramsENet = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
              'l1_ratio': [0.0, 0.5, 1.0, 0.1],
              'max_iter': [1, 5, 10, 100, 1000],}

for ft_dummyfication in [False]: # [False, True]:
    if ft_dummyfication:
        #Creating dummy variables in pandas
        df = pd.get_dummies(df, columns=list(df.drop(columns=["cid"]).drop(columns=size_methods).drop(columns=additional_ft).columns.values))
        df_importance = pd.get_dummies(df_importance, columns=list(df_importance.drop(columns=["nbyes", "nbno", "nbyesmodule"]).columns.values))
        
    ft_selection =  {'None': df.drop(columns=["cid"]).drop(columns=size_methods).columns,
                    # 'RandomForest': df_importance.columns[:100],
                     # 'RandomForest200': df_importance.columns[:200],
                     # 'RandomForest1000': df_importance.columns[:1000],
                    }
        
    for origin_ft_selection in ft_selection:
        print(origin_ft_selection)
        # for size in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
        for size in [0.9]:
            print(size)

            x_train, x_test, y_train, y_test = train_test_split(df[ft_selection[origin_ft_selection]], df[target], test_size=size, random_state=42)  
            # x_train, y_train = np.array(x_train.values), np.array(y_train.values)

            model = {# 'LinearRegression': LinearRegression(),
                    # 'Lasso': GridSearchCV(linear_model.Lasso(), param_grid=paramsLasso, cv=3).fit(x_train, y_train).best_estimator_,
#                     'LassoCV': linear_model.LassoCV(cv=2).fit(x_train, y_train),                    
                     # 'Ridge': GridSearchCV(linear_model.Ridge(), param_grid=paramsRidge, cv=3).fit(x_train, y_train).best_estimator_,
                     # 'RidgeCV': linear_model.RidgeCV(cv=2).fit(x_train, y_train),  
                    # 'ElasticNet': GridSearchCV(ElasticNet(), param_grid=paramsENet, scoring='r2', cv=3).fit(x_train, y_train).best_estimator_,
                    'ElasticNetCV': linear_model.ElasticNetCV().fit(x_train, y_train),
                   # 'PolynomialRegression': LinearRegression()
                    }
        
            for key in model:
                if (key == 'PolynomialRegression') and (origin_ft_selection == 'None'):
                    print("It doesn't scale")
                else:
                    if (key == 'PolynomialRegression'):
                        #for 2 options we have: 𝑓(𝑥₁, 𝑥₂) = 𝑏₀ + 𝑏₁𝑥₁ + 𝑏₂𝑥₂ + 𝑏₃𝑥₁² + 𝑏₄𝑥₁𝑥₂ + 𝑏₅𝑥₂²
                        pf = PolynomialFeatures(degree=2, include_bias=False)
                        x_train_pf = pf.fit_transform(x_train)
                        x_test_pf = pf.fit_transform(x_test)
                        res_model = run_regressorML(model[key], test_size=size, size_target=target, x_train=x_train_pf, x_test=x_test_pf)
                        # model[key].coef_ = model[key].coef_[:100]
                        importanceSeries = pd.Series(model[key].coef_, index=pf.get_feature_names(x_train.columns))
                        coef_order = importanceSeries[importanceSeries != 0].abs().sort_values(ascending = False)
                    else:
                        res_model = run_regressorML(model[key], test_size=size, size_target=target, x_train=x_train, x_test=x_test)
                        coef_order = ft_importances(model[key].coef_, col=ft_selection[origin_ft_selection])
                   
                        
                    df_result.loc[len(df_result)] = [key, ft_dummyfication, origin_ft_selection, ft_selection[origin_ft_selection], model[key], target, additional_ft, size, coef_order, res_model]

df_result.to_csv("results.csv", header=True)

None
0.9


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [10]:
#df_result['coef_order']

In [11]:
res = pd.read_csv('results.csv')
res

,Unnamed: 0,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,additional_ft,test_size,coef_order,accuracy
0,0,ElasticNetCV,False,None,"Index(['OPENVSWITCH', 'TEXTSEARCH_FSM', 'NETFI...","ElasticNetCV(alphas=None, copy_X=True, cv='war...",vmlinux,"['nbyes', 'nbno', 'nbmodule', 'nbyesmodule']",0.9,nbyes 934.629498\nnbmodule 665....,count 83224.000000\nmean 80.177295\n...


In [12]:
# oblige to reparse 
coeffs = res.iloc[0]['coef_order']
coef = pd.DataFrame(columns=['option', 'coefficient'])
i = 0
coeffs.splitlines()
for coeff in coeffs.splitlines():
    print(coeff)
    optcoeff = [c for c in coeff.split(' ') if c is not '']
    if len(optcoeff) == 1:
        continue
    elif len(optcoeff) == 3:
        opt = optcoeff[0] + " " + optcoeff[1]
        coeff = optcoeff[2]
    else: # standard: one option, one coeff
        opt = optcoeff[0]
        coeff = optcoeff[1]
    coef.loc[i] = (opt, coeff)
    i = i + 1
coef

nbyes          934.629498
nbmodule       665.023542
nbyesmodule    268.605829
nbno           268.605815
DEBUG_INFO       0.512936
dtype: float64


,option,coefficient
0,nbyes,934.629498
1,nbmodule,665.023542
2,nbyesmodule,268.605829
3,nbno,268.605815
4,DEBUG_INFO,0.512936
5,dtype:,float64


In [13]:
# Boxplot
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

data = [order_lr, order_lasso, order_ridge, order_en]
fig1, ax1 = plt.subplots()
ax1.set_title('Coeficient of feature importance for all algorithms')
ax1.boxplot(data)
plt.show()

/usr/local/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


NameError: name 'order_lr' is not defined

In [ ]:
res.iloc[1]['hyperparameters']

In [ ]:
# 'nbyes' in coef['option']